In [5]:
import pandas as pd



# Set the display options
#pd.options.display.float_format = '{:,.0f}'.format
#pd.options.display.float_format = None
pd.set_option('display.max_rows', 500)

# PREPROCESSING 1

In [8]:
df=pd.read_parquet('data/transactions.parquet')
print(len(df))

#kill duplicates based on previous analysis
df1 = df.drop_duplicates()
df1.reset_index(drop=True, inplace=True)
print(len(df1))


# create a discount rate and a profit columns
df2=df1.copy()
df2['discount_rate_%'] = 100*(df1['OriginalSaleAmountInclVAT']-df1['RevenueInclVAT']) / df1['OriginalSaleAmountInclVAT']
df2['profit_%'] = 100*(df1['RevenueInclVAT']-df1['CostPriceExclVAT']) / df1['OriginalSaleAmountInclVAT']
df2['Order_Date_FK'] = pd.to_datetime(df2['Order_Date_FK'], format='%Y%m%d')
df2['day_of_week'] = df2['Order_Date_FK'].dt.day_name()


#reorder columns
df2 = df2[[ 'SaleDocumentNumber','Order_Date_FK', 'day_of_week', 'Shop', 'CustomerID', 'ProductCode', 'OriginalSaleAmountInclVAT', 'RevenueInclVAT', 'discount_rate_%','CostPriceExclVAT', 'profit_%', 'BrandName', 'ModelGroup', 'ProductGroup', 'Returned']]

# Convert day in week to numerical with dummies and ensure output is integer
df3 = pd.get_dummies(df2, columns=['day_of_week'], drop_first=True)
dummy_columns = [col for col in df3.columns if 'day_of_week' in col]
df3[dummy_columns] = df3[dummy_columns].astype('int64')

# add the number of item in the transaction
df3['nb_item_tx'] = df3.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df3['nb_similar_item_tx'] = df3.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df4 = df3.drop(columns=['Order_Date_FK', 'SaleDocumentNumber', 'RevenueInclVAT', 'CostPriceExclVAT', 'ProductGroup', 'ModelGroup'])
display(df4.head(5))



df4.to_parquet('data/data_clean.parquet')

1759889
943354


,Shop,CustomerID,ProductCode,OriginalSaleAmountInclVAT,discount_rate_%,profit_%,BrandName,Returned,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,nb_item_tx,nb_similar_item_tx
0,36,-2190786785520839526,-1968361059464632550,99.95,25.002501,38.444707,3694837121284491212,0,0,0,0,0,0,0,1,1
1,58,-3499949798555834600,-1745497788861904687,21.95,0.000000,66.448642,-1657270726128381662,0,0,0,0,0,0,0,1,1
2,49,-7841417233849937172,-4482110002029860032,69.95,45.003574,20.092924,-7202472198306336051,0,0,0,0,0,0,0,2,1
3,49,-7841417233849937172,-9146072753857637799,99.95,45.002501,22.687838,-2074012670196538378,0,0,0,0,0,0,0,2,1
4,15,7033964597104594686,-8829485741699345914,49.95,65.005005,17.205942,-4869459690934553786,0,0,0,0,0,0,0,2,1


# PREPROCESSING 2  - KEEP DUPLICATES

In [5]:
df=pd.read_parquet('data/transactions.parquet')
print(len(df))

# #kill duplicates based on previous analysis
# df1 = df.drop_duplicates()
# df1.reset_index(drop=True, inplace=True)
# print(len(df1))


# create a discount rate and a profit columns
df2=df.copy()
df2['discount_rate_%'] = 100*(df['OriginalSaleAmountInclVAT']-df['RevenueInclVAT']) / df['OriginalSaleAmountInclVAT']
df2['profit_%'] = 100*(df['RevenueInclVAT']-df['CostPriceExclVAT']) / df['OriginalSaleAmountInclVAT']
df2['Order_Date_FK'] = pd.to_datetime(df2['Order_Date_FK'], format='%Y%m%d')
df2['day_of_week'] = df2['Order_Date_FK'].dt.day_name()


#reorder columns
df2 = df2[[ 'SaleDocumentNumber','Order_Date_FK', 'day_of_week', 'Shop', 'CustomerID', 'ProductCode', 'OriginalSaleAmountInclVAT', 'RevenueInclVAT', 'discount_rate_%','CostPriceExclVAT', 'profit_%', 'BrandName', 'ModelGroup', 'ProductGroup', 'Returned']]

# Convert day in week to numerical with dummies and ensure output is integer
df3 = pd.get_dummies(df2, columns=['day_of_week'], drop_first=True)
dummy_columns = [col for col in df3.columns if 'day_of_week' in col]
df3[dummy_columns] = df3[dummy_columns].astype('int64')


# add the number of item in the transaction
df3['nb_item_tx'] = df3.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df3['nb_similar_item_tx'] = df3.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df4 = df3.drop(columns=['Order_Date_FK', 'SaleDocumentNumber', 'RevenueInclVAT', 'CostPriceExclVAT', 'ProductGroup', 'ModelGroup'])
display(df4.head())

print(len(df4))



df4.to_parquet('data/data_clean2.parquet')

1759889


,Shop,CustomerID,ProductCode,OriginalSaleAmountInclVAT,discount_rate_%,profit_%,BrandName,Returned,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,nb_item_tx,nb_similar_item_tx
0,36,-2190786785520839526,-1968361059464632550,99.95,25.002501,38.444707,3694837121284491212,0,0,0,0,0,0,0,2,2
1,58,-3499949798555834600,-1745497788861904687,21.95,0.000000,66.448642,-1657270726128381662,0,0,0,0,0,0,0,2,2
2,49,-7841417233849937172,-4482110002029860032,69.95,45.003574,20.092924,-7202472198306336051,0,0,0,0,0,0,0,4,2
3,49,-7841417233849937172,-9146072753857637799,99.95,45.002501,22.687838,-2074012670196538378,0,0,0,0,0,0,0,4,2
4,15,7033964597104594686,-8829485741699345914,49.95,65.005005,17.205942,-4869459690934553786,0,0,0,0,0,0,0,6,2


1759889


# PREPROCESSING 3 - Insert Model/group column back 

In [7]:
df=pd.read_parquet('data/transactions.parquet')
print(len(df))

#kill duplicates based on previous analysis
df1 = df.drop_duplicates()
df1.reset_index(drop=True, inplace=True)
print(len(df1))


# create a discount rate and a profit columns
df2=df.copy()
df2['discount_rate_%'] = 100*(df['OriginalSaleAmountInclVAT']-df['RevenueInclVAT']) / df['OriginalSaleAmountInclVAT']
df2['profit_%'] = 100*(df['RevenueInclVAT']-df['CostPriceExclVAT']) / df['OriginalSaleAmountInclVAT']
df2['Order_Date_FK'] = pd.to_datetime(df2['Order_Date_FK'], format='%Y%m%d')
df2['day_of_week'] = df2['Order_Date_FK'].dt.day_name()


#reorder columns
df2 = df2[[ 'SaleDocumentNumber','Order_Date_FK', 'day_of_week', 'Shop', 'CustomerID', 'ProductCode', 'OriginalSaleAmountInclVAT', 'RevenueInclVAT', 'discount_rate_%','CostPriceExclVAT', 'profit_%', 'BrandName', 'ModelGroup', 'ProductGroup', 'Returned']]

# Convert day in week to numerical with dummies and ensure output is integer
df3 = pd.get_dummies(df2, columns=['day_of_week'], drop_first=True)
dummy_columns = [col for col in df3.columns if 'day_of_week' in col]
df3[dummy_columns] = df3[dummy_columns].astype('int64')


# add the number of item in the transaction
df3['nb_item_tx'] = df3.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df3['nb_similar_item_tx'] = df3.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df4 = df3.drop(columns=['Order_Date_FK', 'SaleDocumentNumber', 'RevenueInclVAT', 'CostPriceExclVAT'])
display(df4.head())



df4.to_parquet('data/data_clean3.parquet')

1759889
943354


,Shop,CustomerID,ProductCode,OriginalSaleAmountInclVAT,discount_rate_%,profit_%,BrandName,ModelGroup,ProductGroup,Returned,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,nb_item_tx,nb_similar_item_tx
0,36,-2190786785520839526,-1968361059464632550,99.95,25.002501,38.444707,3694837121284491212,3162564956579801398,-453682476182549203,0,0,0,0,0,0,0,2,2
1,58,-3499949798555834600,-1745497788861904687,21.95,0.000000,66.448642,-1657270726128381662,-6715310361529623819,-453682476182549203,0,0,0,0,0,0,0,2,2
2,49,-7841417233849937172,-4482110002029860032,69.95,45.003574,20.092924,-7202472198306336051,-248459121805647149,5294166889150293396,0,0,0,0,0,0,0,4,2
3,49,-7841417233849937172,-9146072753857637799,99.95,45.002501,22.687838,-2074012670196538378,-248459121805647149,5294166889150293396,0,0,0,0,0,0,0,4,2
4,15,7033964597104594686,-8829485741699345914,49.95,65.005005,17.205942,-4869459690934553786,-4124706989267335995,-453682476182549203,0,0,0,0,0,0,0,6,2


# PREPROC 4 : Put back invoice nb and other numerical columns

In [27]:
df=pd.read_parquet('data/transactions.parquet')
print(len(df))

#kill duplicates based on previous analysis
df1 = df.drop_duplicates()
df1.reset_index(drop=True, inplace=True)
print(len(df1))


# create a discount rate and a profit columns
df2=df1.copy()
df2['discount_rate_%'] = 100*(df1['OriginalSaleAmountInclVAT']-df1['RevenueInclVAT']) / df1['OriginalSaleAmountInclVAT']
df2['profit_%'] = 100*(df1['RevenueInclVAT']-df1['CostPriceExclVAT']) / df1['OriginalSaleAmountInclVAT']
df2['Order_Date_FK'] = pd.to_datetime(df2['Order_Date_FK'], format='%Y%m%d')
df2['day_of_week'] = df2['Order_Date_FK'].dt.day_name()


#reorder columns
df2 = df2[[ 'SaleDocumentNumber','Order_Date_FK', 'day_of_week', 'Shop', 'CustomerID', 'ProductCode', 'OriginalSaleAmountInclVAT', 'RevenueInclVAT', 'discount_rate_%','CostPriceExclVAT', 'profit_%', 'BrandName', 'ModelGroup', 'ProductGroup', 'Returned']]

# Convert day in week to numerical with dummies and ensure output is integer
df3 = pd.get_dummies(df2, columns=['day_of_week'], drop_first=True)
dummy_columns = [col for col in df3.columns if 'day_of_week' in col]
df3[dummy_columns] = df3[dummy_columns].astype('int64')


# add the number of item in the transaction
df3['nb_item_tx'] = df3.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df3['nb_similar_item_tx'] = df3.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df4 = df3.drop(columns=[])
display(df4.head())
print(len(df4))



df4.to_parquet('data/data_clean4.parquet')

1759889
943354


,SaleDocumentNumber,Order_Date_FK,Shop,CustomerID,ProductCode,OriginalSaleAmountInclVAT,RevenueInclVAT,discount_rate_%,CostPriceExclVAT,profit_%,...,ProductGroup,Returned,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,nb_item_tx,nb_similar_item_tx
0,23995792,2021-07-30,36,-2190786785520839526,-1968361059464632550,99.95,74.96,25.002501,36.534515,38.444707,...,-453682476182549203,0,0,0,0,0,0,0,1,1
1,23995794,2021-07-30,58,-3499949798555834600,-1745497788861904687,21.95,21.95,0.000000,7.364523,66.448642,...,-453682476182549203,0,0,0,0,0,0,0,1,1
2,23995797,2021-07-30,49,-7841417233849937172,-4482110002029860032,69.95,38.47,45.003574,24.415000,20.092924,...,5294166889150293396,0,0,0,0,0,0,0,2,1
3,23995797,2021-07-30,49,-7841417233849937172,-9146072753857637799,99.95,54.97,45.002501,32.293506,22.687838,...,5294166889150293396,0,0,0,0,0,0,0,2,1
4,23995798,2021-07-30,15,7033964597104594686,-8829485741699345914,49.95,17.48,65.005005,8.885632,17.205942,...,-453682476182549203,0,0,0,0,0,0,0,2,1


943354


# PREPROC 5 - put back DUPLICATES

In [2]:
df=pd.read_parquet('data/transactions.parquet')
print(len(df))

# #kill duplicates based on previous analysis
# df1 = df.drop_duplicates()
# df1.reset_index(drop=True, inplace=True)
# print(len(df1))


# create a discount rate and a profit columns
df2=df.copy()
df2['discount_rate_%'] = 100*(df['OriginalSaleAmountInclVAT']-df['RevenueInclVAT']) / df['OriginalSaleAmountInclVAT']
df2['profit_%'] = 100*(df['RevenueInclVAT']-df['CostPriceExclVAT']) / df['OriginalSaleAmountInclVAT']
df2['Order_Date_FK'] = pd.to_datetime(df2['Order_Date_FK'], format='%Y%m%d')
df2['day_of_week'] = df2['Order_Date_FK'].dt.day_name()


#reorder columns
df2 = df2[[ 'SaleDocumentNumber','Order_Date_FK', 'day_of_week', 'Shop', 'CustomerID', 'ProductCode', 'OriginalSaleAmountInclVAT', 'RevenueInclVAT', 'discount_rate_%','CostPriceExclVAT', 'profit_%', 'BrandName', 'ModelGroup', 'ProductGroup', 'Returned']]

# Convert day in week to numerical with dummies and ensure output is integer
df3 = pd.get_dummies(df2, columns=['day_of_week'], drop_first=True)
dummy_columns = [col for col in df3.columns if 'day_of_week' in col]
df3[dummy_columns] = df3[dummy_columns].astype('int64')


# add the number of item in the transaction
df3['nb_item_tx'] = df3.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df3['nb_similar_item_tx'] = df3.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df4 = df3.drop(columns=[])
display(df4.head())
print(len(df4))



df4.to_parquet('data/data_clean5.parquet')

1759889


,SaleDocumentNumber,Order_Date_FK,Shop,CustomerID,ProductCode,OriginalSaleAmountInclVAT,RevenueInclVAT,discount_rate_%,CostPriceExclVAT,profit_%,...,ProductGroup,Returned,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,nb_item_tx,nb_similar_item_tx
0,23995792,2021-07-30,36,-2190786785520839526,-1968361059464632550,99.95,74.96,25.002501,36.534515,38.444707,...,-453682476182549203,0,0,0,0,0,0,0,2,2
1,23995794,2021-07-30,58,-3499949798555834600,-1745497788861904687,21.95,21.95,0.000000,7.364523,66.448642,...,-453682476182549203,0,0,0,0,0,0,0,2,2
2,23995797,2021-07-30,49,-7841417233849937172,-4482110002029860032,69.95,38.47,45.003574,24.415000,20.092924,...,5294166889150293396,0,0,0,0,0,0,0,4,2
3,23995797,2021-07-30,49,-7841417233849937172,-9146072753857637799,99.95,54.97,45.002501,32.293506,22.687838,...,5294166889150293396,0,0,0,0,0,0,0,4,2
4,23995798,2021-07-30,15,7033964597104594686,-8829485741699345914,49.95,17.48,65.005005,8.885632,17.205942,...,-453682476182549203,0,0,0,0,0,0,0,6,2


1759889


# PREPROC 6 - simplify dummies vs 1hot in the main

In [4]:
import pandas as pd

#  load orginal data to df
df=pd.read_parquet('data/transactions.parquet')
print(df.shape)
print(df.columns)


# create a discount rate and a profit and day of the week columns

df['discount_rate_%'] = 100*(df['OriginalSaleAmountInclVAT']-df['RevenueInclVAT']) / df['OriginalSaleAmountInclVAT']
df['profit_%'] = 100*(df['RevenueInclVAT']-df['CostPriceExclVAT']) / df['OriginalSaleAmountInclVAT']
df['Order_Date_FK'] = pd.to_datetime(df['Order_Date_FK'], format='%Y%m%d')
df['day_of_week'] = df['Order_Date_FK'].dt.day_name()

# add the number of item in the transaction
df['nb_item_tx'] = df.groupby('SaleDocumentNumber')['ProductCode'].transform('count')

# add the number of similar item in the  transaction
df['nb_similar_item_tx'] = df.groupby(['SaleDocumentNumber', 'ProductCode'])['ProductCode'].transform('count')


#drop some columns that are not useful for Machine Learning :
df = df.drop(columns=[])
display(df.head())
print(df.shape)


df.to_parquet('data/data_clean6.parquet')

(1759889, 12)
Index(['Shop', 'Order_Date_FK', 'ProductCode', 'OriginalSaleAmountInclVAT',
       'CustomerID', 'SaleDocumentNumber', 'Returned', 'RevenueInclVAT',
       'CostPriceExclVAT', 'BrandName', 'ModelGroup', 'ProductGroup'],
      dtype='object')


,Shop,Order_Date_FK,ProductCode,OriginalSaleAmountInclVAT,CustomerID,SaleDocumentNumber,Returned,RevenueInclVAT,CostPriceExclVAT,BrandName,ModelGroup,ProductGroup,discount_rate_%,profit_%,day_of_week,nb_item_tx,nb_similar_item_tx
0,36,2021-07-30,-1968361059464632550,99.95,-2190786785520839526,23995792,0,74.96,36.534515,3694837121284491212,3162564956579801398,-453682476182549203,25.002501,38.444707,Friday,2,2
1,58,2021-07-30,-1745497788861904687,21.95,-3499949798555834600,23995794,0,21.95,7.364523,-1657270726128381662,-6715310361529623819,-453682476182549203,0.000000,66.448642,Friday,2,2
2,49,2021-07-30,-4482110002029860032,69.95,-7841417233849937172,23995797,0,38.47,24.415000,-7202472198306336051,-248459121805647149,5294166889150293396,45.003574,20.092924,Friday,4,2
3,49,2021-07-30,-9146072753857637799,99.95,-7841417233849937172,23995797,0,54.97,32.293506,-2074012670196538378,-248459121805647149,5294166889150293396,45.002501,22.687838,Friday,4,2
4,15,2021-07-30,-8829485741699345914,49.95,7033964597104594686,23995798,0,17.48,8.885632,-4869459690934553786,-4124706989267335995,-453682476182549203,65.005005,17.205942,Friday,6,2


(1759889, 17)
